In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sklearn
from imblearn.over_sampling import RandomOverSampler

In [2]:
# path of dataset
data_path = "./drive/MyDrive/Colab Notebooks/FYP Model/data/Appliance Detection Dataset/Training Dataset/"
data_arrays_path = "./drive/MyDrive/Colab Notebooks/FYP Model/data_arrays/Appliance Detection Dataset/Training Dataset/"

# assigning parameters for MFCC feature extraction
max_len_nfft = 649
n_mfcc = 128

In [3]:
# Saving data to array file
save_data_to_array(data_path=data_path, data_arrays_path=data_arrays_path, max_len=max_len_nfft, n_mfcc=n_mfcc)

Saving vectors of label - 'Bulb': 100%|██████████| 3530/3530 [01:57<00:00, 30.10it/s]
Saving vectors of label - 'Appliance Detection Negative': 100%|██████████| 1185/1185 [00:26<00:00, 44.97it/s]


Streaming output truncated to the last 5000 lines.
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
223
224
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
226
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
227
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
228
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
2

In [4]:
# Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test(data_path=data_path, data_arrays_path=data_arrays_path, split_ratio=0.8)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (6204, 128, 649)
X_test:  (1552, 128, 649)


In [5]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# reshaping the data for oversampling
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

# fit and apply the transform
X_train, y_train = oversample.fit_resample(X_train, y_train)
X_test, y_test = oversample.fit_resample(X_test, y_test)

# converting it back to original shape
X_train = X_train.reshape(X_train.shape[0], n_mfcc, max_len_nfft)
X_test = X_test.reshape(X_test.shape[0], n_mfcc, max_len_nfft)

print("X_train: ", X_train.shape, end='\t')
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape, end='\t')
print("y_test: ", y_test.shape)

X_train:  (8647, 128, 649)	y_train:  (8647,)
X_test:  (2132, 128, 649)	y_test:  (2132,)


In [6]:
# Feature dimension
epochs = 50
batch_size = 64
num_classes = 5

# Got help in building model from: https://mikesmales.medium.com/sound-classification-using-deep-learning-8bc2aa1990b7
# model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(n_mfcc, max_len_nfft, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 648, 16)      80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 324, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 324, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 323, 32)       2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 161, 32)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 161, 32)       0

In [7]:
# adding loss and optimizer
model.compile(loss="SparseCategoricalCrossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [8]:
# training the model
model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/50
136/136 [==============================] - 35s 164ms/step - loss: 1.3905 - accuracy: 0.3748 - val_loss: 1.2825 - val_accuracy: 0.5488
Epoch 2/50
136/136 [==============================] - 21s 154ms/step - loss: 1.1320 - accuracy: 0.5551 - val_loss: 1.1005 - val_accuracy: 0.6079
Epoch 3/50
136/136 [==============================] - 21s 156ms/step - loss: 0.9841 - accuracy: 0.6239 - val_loss: 0.9394 - val_accuracy: 0.6745
Epoch 4/50
136/136 [==============================] - 22s 158ms/step - loss: 0.8775 - accuracy: 0.6739 - val_loss: 0.8339 - val_accuracy: 0.7059
Epoch 5/50
136/136 [==============================] - 22s 158ms/step - loss: 0.8074 - accuracy: 0.6952 - val_loss: 0.9262 - val_accuracy: 0.6557
Epoch 6/50
136/136 [==============================] - 22s 160ms/step - loss: 0.7276 - accuracy: 0.7226 - val_loss: 0.7185 - val_accuracy: 0.7444
Epoch 7/50
136/136 [==============================] - 22s 161ms/step - loss: 0.6819 - accuracy: 0.7477 - val_loss: 0.7348 - val_ac

In [9]:
# saving the model
model.save("./drive/MyDrive/Colab Notebooks/FYP Model/Models/Appliance Detection Model")

INFO:tensorflow:Assets written to: ./drive/MyDrive/Colab Notebooks/FYP Model/Models/Appliance Detection Model/assets
